<a href="https://colab.research.google.com/github/kikun1234/Aussie_Suburb_Scanner/blob/main/GitHub_Suburb_Scanner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importing libraries 
import json
import requests 
import time
import pandas as pd

#defining function that accesses Domain API. This function finds selling price of homes.
#creating GET POST function that gets data
def domain_Sale(x,y):
  response = requests.post('https://auth.domain.com.au/v1/connect/token', 
                         data = {'client_id':"######",
                                 "client_secret":"#####",
                                 "grant_type":"client_credentials",
                                 "scope":"api_listings_read",
                                 "Content-Type":"text/json"})
  token=response.json()
  access_token=token["access_token"]
  auth = {"Authorization":"Bearer "+access_token}
  url = "https://api.domain.com.au/v1/listings/residential/_search"
  post_fields ={
    "listingType":"Sale",
    "minPrice":"",
    "pageSize":200,
    "propertyTypes":[
                     "House"
                     ],
    "minBedrooms": "",
    "minBathrooms": "",
    "locations":[
                 {
          "state":x,
          "region":"",
          "area":"",
          "suburb":y,
          "postCode":"",
          "includeSurroundingSuburbs":False
        }
      ]
    }

  #calling API
  request = requests.post(url,headers=auth,json=post_fields)

  #manipulation of JSON data from NoSQL Database
  l=request.json()
  Propertydetails=[]
  for listing in l:
    Propertydetails.append(listing["listing"]["propertyDetails"])
  prop = pd.json_normalize(Propertydetails)

  Pricedetails=[]
  for listing in l:
    Pricedetails.append(listing["listing"]["priceDetails"])
  price = pd.json_normalize(Pricedetails)


  df = prop.join(price)
  df['landAreaNormal'] = df['landArea']/100

  #importing data cleansing function from sklearn
  from sklearn.preprocessing import MultiLabelBinarizer
  mlb = MultiLabelBinarizer()
  df = df.join(pd.DataFrame(mlb.fit_transform(df.pop('features')),
                          columns=mlb.classes_,
                          index=df.index))

  return(df)

In [ ]:
#same as above except this function finds sold prices of homes
import json
import requests 
import time
import pandas as pd

def domain_Sold(x,y):
  response = requests.post('https://auth.domain.com.au/v1/connect/token', 
                         data = {'client_id':"####",
                                 "client_secret":"#####",
                                 "grant_type":"client_credentials",
                                 "scope":"api_listings_read",
                                 "Content-Type":"text/json"})
  token=response.json()
  access_token=token["access_token"]
  auth = {"Authorization":"Bearer "+access_token}
  url = "https://api.domain.com.au/v1/listings/residential/_search"
  post_fields ={
    "listingType":"Sold",
    "minPrice":"",
    "pageSize":200,
    "propertyTypes":[
                     "House"
                     ],
    "minBedrooms": "",
    "minBathrooms": "",
    "locations":[
                 {
          "state":x,
          "region":"",
          "area":"",
          "suburb":y,
          "postCode":"",
          "includeSurroundingSuburbs":False
        }
      ]
    }

  request = requests.post(url,headers=auth,json=post_fields)

  l=request.json()
  Propertydetails=[]
  for listing in l:
    Propertydetails.append(listing["listing"]["propertyDetails"])
  prop = pd.json_normalize(Propertydetails)

  Pricedetails=[]
  for listing in l:
    Pricedetails.append(listing["listing"]["priceDetails"])
  price = pd.json_normalize(Pricedetails)

  Solddata=[]
  for listing in l:
    Solddata.append(listing["listing"]["soldData"])
  sold = pd.json_normalize(Solddata)

  df = prop.join(price)
  df = df.join(sold)
  df['landAreaNormal'] = df['landArea']/100
  df['soldDateNormal'] = (pd.to_datetime(df['soldDate']).dt.year - 2000)/10

  from sklearn.preprocessing import MultiLabelBinarizer
  mlb = MultiLabelBinarizer()
  df = df.join(pd.DataFrame(mlb.fit_transform(df.pop('features')),
                          columns=mlb.classes_,
                          index=df.index))

  return(df)

In [ ]:
#defining kmeans algorithm from sklearn
def cluster(x):
  from sklearn.cluster import KMeans
  y = x[['latitude','longitude']]
  kmeans = KMeans(n_clusters=3, random_state=0).fit(y)
  x['Cluster'] = kmeans.labels_
  return(x)

In [ ]:
 #running functions for sold homes and Kmeans algorithm
 sb = domain_Sold("QLD", "Sunnybank Hills")
 sb = cluster(sb)

In [ ]:
#example data set of sold prices
sb[sb.soldPrice < 400000]

,state,propertyType,allPropertyTypes,bathrooms,bedrooms,carspaces,unitNumber,streetNumber,street,area,...,RainwaterStorageTank,SecureParking,SeparateDiningRoom,Shed,SolarHotWater,SolarPanels,Study,SwimmingPool,TennisCourt,Cluster
13,QLD,House,[House],1.0,2.0,1,46,452,Hellawell Road,Brisbane Region,...,0,0,0,0,0,0,0,0,0,0
109,QLD,House,[House],1.0,3.0,2,,24,Benbek Circuit,Brisbane Region,...,0,0,0,0,0,0,0,0,1,0
134,QLD,House,[House],1.0,3.0,2,,501,Beenleigh Road,Brisbane Region,...,0,1,0,0,0,0,0,1,0,0
177,QLD,House,[House],1.0,3.0,1,,165,Ryhill Road,Brisbane Region,...,0,0,0,0,0,1,0,0,0,2
189,QLD,House,[House],1.0,3.0,1,,96,Ditton Road,Brisbane Region,...,0,1,0,1,0,0,0,0,0,2
190,QLD,House,[House],1.0,3.0,2,,96,Morden Road,Brisbane Region,...,0,0,0,0,0,0,0,0,0,2


In [ ]:
#running function for homes for sale
sb_sale = domain_Sale("QLD", "Sunnybank Hills")

In [ ]:
#example of data for some homes
sb_sale

,state,propertyType,allPropertyTypes,bathrooms,bedrooms,carspaces,unitNumber,streetNumber,street,area,...,PetsAllowed,RainwaterStorageTank,SecureParking,SeparateDiningRoom,Shed,SolarHotWater,SolarPanels,Study,SwimmingPool,WaterEfficientAppliances
0,QLD,House,[House],3.0,4.0,2,,11,Ashwood Court,Brisbane Region,...,0,1,1,0,1,1,1,1,1,0
1,QLD,House,[House],3.0,5.0,4,,16,Jales Street,Brisbane Region,...,0,0,0,0,0,0,0,0,0,0
2,QLD,House,[House],3.0,5.0,2,,78,Stones Road,Brisbane Region,...,0,0,1,0,0,0,0,0,0,0
3,QLD,House,[House],1.0,3.0,2,,50,Jackson Road,Brisbane Region,...,0,0,0,0,1,0,0,0,0,0
4,QLD,House,[House],3.0,5.0,2,,19,Peatmoss Street,Brisbane Region,...,0,0,1,0,0,0,0,0,1,0
5,QLD,House,[House],2.0,4.0,2,,27,Tallowood Way,Brisbane Region,...,0,0,1,0,0,0,0,0,0,0
6,QLD,House,[House],2.0,4.0,2,,54,Hellawell Road,Brisbane Region,...,0,0,0,0,0,0,0,0,1,0
7,QLD,House,[House],3.0,4.0,2,,2,Westminster Place,Brisbane Region,...,0,0,0,0,1,0,0,0,0,0
8,QLD,House,[House],3.0,5.0,2,,11,Tanguile Place,Brisbane Region,...,0,1,1,0,1,0,0,1,1,0
9,QLD,House,[House],2.0,3.0,2,,12,Manzill Street,Brisbane Region,...,0,0,0,0,1,0,0,1,0,0
